# External Data
## CityBike
Write a function that lists all the nearest citybike stations to a given location.

When starting a project like this it is useful to think about the steps required to arrive at the final solution.
in the case of the citybike example, this could look something like this:
* get the citybike data from the web
    * in particular, get the location of the stations
* calculate the distances from the station locations to a given point
* wrap and format everything nicely

### Exploring the data
first, let's check out the available data on citybikes to see what we are working with. You can also look at this in a webbrowser
but beware: [Humans should never gaze upon unfiltered data!](https://rackandstack-tech.blog/2019/11/12/the-mental-game-of-python-raymond-hettinger/)


http://dynamisch.citybikewien.at/citybike_xml.php?json

In [133]:
import requests

data = requests.get(url="http://dynamisch.citybikewien.at/citybike_xml.php?json")
data, type(data)

(<Response [200]>, requests.models.Response)

whats that Response object? what can we do with it?

we can look in the [documentation here]() or just look at the ```help(on_something_please)``` directly

In [131]:
help(data)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

In [134]:
data.headers['content-type']

'application/json;charset=utf-8'

The url *AND* header info on the content type suggest that the response can in fact be interpreted as json format.

As the old saying goes:
> if it looks like a duck
>
> and quacks like a duck...
>
> it probably is a duck

So let's try interpreting the response as json:

In [139]:
cb_json = data.json()
print(f"{type(cb_json)=}\n{len(cb_json)=}")

type(cb_json)=<class 'list'>
len(cb_json)=119


so the json interpretation yields us a list with 119 entries.let's see what they hold

In [140]:
print("example: Erstes Element: cb_json[0]")
cb_json[0]

example: Erstes Element: cb_json[0]


{'id': 108,
 'name': 'Friedrich Schmidtplatz',
 'status': 'aktiv',
 'description': 'Ecke Lichtenfelsgasse U2 Station Rathaus',
 'boxes': 24,
 'free_boxes': 23,
 'free_bikes': 1,
 'longitude': 16.356581,
 'latitude': 48.211433,
 'internal_id': 1026}

Perfect! Each element in the list is a dictionary, representing the data on a particular city bike station.
Apart from the name, status, description and how many bikes and boxes are available, we also have the positional information
on the location of the stations as latitude and longitude coordinates

To use this, we first need a way to calculate the distance between two locations.
So let's write a function, that takes two positions and returns their distance:

In [34]:
from math import sqrt

def distance(a: tuple, b: tuple):
    """
    Returns the 2D distance of 2 points represented as tuples
    :param a: tuple(x: float, y: float)
    :param b: tuple(x: float, y: float
    :return:  Distance
    """
    dx = abs(a[0] - b[0])
    dy = abs(a[1] - b[1])
    return sqrt(dx**2 + dy**2)

Let's do some tests

In [30]:
distance((0,4), (3,0))

5.0

In [32]:
distance((0,-4), (-3,0))

5.0

In [39]:
station1 = (cb_json[0]["longitude"], cb_json[0]["latitude"])
station2 = (cb_json[1]["longitude"], cb_json[1]["latitude"])
station1, station2

((16.356581, 48.211433), (16.37298586854365, 48.210666083418))

In [40]:
distance(station1, station2)

0.016422785177255206

It works, but what does that actually mean?

What units is the result in? Give it a minute of thought

### Distances on earth
Since latitude and longitude are actually angles on the sphere of the earth,
we need to take this into account.
We actually need to convert the difference in latitude and longitude in meters on the surface.
you remember your sin/cos and the radius of the earth, right?

Also, the earth is not actually a sphere, but an ellipsoid. That means that the closer we are to the poles,
the bigger the actual distance on the surface per angle.

So the question is: How do we calculate a distance on the earth between two coordinates?

This is where the open source nature and the rich ecosystem of python really shine:
As with many other common problems, there sure is a third party library by someone who has already
solved the problem. And much better, safer and more precise than we ever could.

Quick google search "python distance latitude longitude" reveals an appropriate answer:
https://geopy.readthedocs.io/en/stable/#

In [41]:
import geopy

This import statement will most likely cause an error with your vanilla python installation.
It is a third party library that first needs installation.
There are many ways of installing it, the easiest is probably:

Open a terminal/command line in your system and simply type:
```terminal
> pip install geopy
```
You should see some download bars and a success message. More on ```pip install``` here: https://realpython.com/what-is-pip/

Once that works, let's test the module

In [45]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.distance(coords_1, coords_2))

279.35290160430094 km


Perfect, now let's try it with the citybike data

In [49]:
geopy.distance.distance(station1, station2)

Distance(1.8172514919453264)

Now let's define a  function that takes a location and calculate the distance to each station

In [115]:
def list_nearest(location: tuple, list_of_stations):
    for station in list_of_stations:
        station_location = (station["latitude"], station["longitude"])
        dist = geopy.distance.distance(location, station_location)
        #save the ddistance in the station
        station["distance"] = round(dist.km, 1)
    list_of_stations.sort(key=lambda d: d["distance"], reverse=False)
    return list_of_stations


Coordinates of the Energybase are
48.2696523, 16.4271389 according to google maps

And here's the nearest CityBike Stations:

In [84]:
energybase = (48.2696523, 16.4271389)
result = list_nearest(energybase)
for i, station in enumerate(result):
    #help with enumerate: https://realpython.com/python-enumerate/
    print(f"{i+1:3}. [{station['distance']:2.1f} km] {station['name'][:20]:20s} - {station['free_bikes']:2}/{station['boxes']}")
    #help on format strings: https://realpython.com/python-f-strings/#f-strings-a-new-and-improved-way-to-format-strings-in-python

  1. [4.4 km] Millennium Tower     - 24/35
  2. [4.6 km] Friedrich Engels Pla - 22/27
  3. [5.0 km] Hellwagstraße        -  8/30
  4. [5.1 km] Traisengasse         - 17/24
  5. [5.6 km] Radingerstraße       - 27/33
  6. [5.7 km] Jägerstraße U6       - 11/26
  7. [6.0 km] Messeplatz           -  5/40
  8. [6.0 km] Wallensteinplatz     - 16/27
  9. [6.2 km] Krieau               - 10/24
 10. [6.3 km] Heinestraße          - 14/38
 11. [6.3 km] Praterstern          - 14/40
 12. [6.4 km] Spittelau U4/U6      -  8/26
 13. [6.5 km] Südportalstraße      - 21/40
 14. [6.5 km] Novaragasse          - 15/24
 15. [6.6 km] Friedensbrücke       -  3/26
 16. [6.6 km] Stadion Center       - 17/40
 17. [6.7 km] Nepomukgasse         -  9/15
 18. [6.8 km] Althanstraße         -  3/22
 19. [6.8 km] Roßauer Lände U4     - 15/28
 20. [6.9 km] Julius-Tandler-Platz - 14/26
 21. [6.9 km] Gymnasiumstraße      - 19/26
 22. [6.9 km] Nussdorfer Straße U6 -  3/19
 23. [7.0 km] Karmeliterplatz      -  3/14
 24. [7.0 k

Now let's check if this actually correct:
https://www.google.com/search?client=firefox-b-d&tbs=lf:1,lf_ui:3&tbm=lcl&q=city+bike+terminal&rflfq=1&num=10&ved=2ahUKEwik7o_j3tPzAhXBlIsKHZHfCEcQtgN6BAgNEAQ#rlfi=hd:;si:5765786581731636286;mv:[[48.24762833716234,16.404557336422684],[48.235752855295445,16.375804055783036],null,[48.24169094084712,16.39018069610286],16]

### Addresses as inputs
Another thing that is still kinda messy, is that we need to come up with coordinates
ourselves by looking up the address in google maps.
Wouldn't it be much nicer, if we could just input an address to python directly, and it fetches the coordinates for us?

It is relatively easy using open streetmaps

#%

In [104]:

address="Giefinggasse 6, 1020 Wien"
addurl = address.replace(" ", "%20")
addurl = addurl.replace(",", "%2C")
addurl

'Giefinggasse%206%2C%201020%20Wien'

the replacement of strings is necessary for the string to work as an url

easier way to do it is using a dedicated library like such:

In [106]:
import urllib.parse
addurl = urllib.parse.quote(address)
addurl

'Giefinggasse%206%2C%201020%20Wien'

In [109]:
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(addurl) +'?format=json'

response = requests.get(url).json()
location = (response[0]["lat"], response[0]["lon"])
location

('48.2696395', '16.427355625558313')

let's use this to create a function that takes an address and returns a location tuple

In [121]:
def from_address(address_string="Stephansplatz, Wien, Österreich"):
    parsed_address = urllib.parse.quote(address_string)
    url = 'https://nominatim.openstreetmap.org/search/' + parsed_address +'?format=json'
    response = requests.get(url).json()
    location = (response[0]["lat"], response[0]["lon"])
    return location

from_address("Giefinggasse 6, 1020 Wien, Österreich")

('48.2696395', '16.427355625558313')

### Putting it all together
Now let's create a function that puts all of this together:
1. fetch the current citybike data
2. calculates the

some further sueful material:
https://towardsdatascience.com/geocode-with-python-161ec1e62b89


In [125]:
def closest_citybikes(address="Stephansplatz, Wien, Österreich", num_results=10):
    cb_data = requests.get(url="http://dynamisch.citybikewien.at/citybike_xml.php?json")
    cb_json = cb_data.json()

    location = from_address(address)
    result = list_nearest(location, list_of_stations=cb_json)

    print(f"{num_results} nearest Stations to {address}:")
    print("_"*len(address))
    for i, station in enumerate(result[:num_results]):
        print(f"{i+1:3}. [{station['distance']:2.1f} km] {station['name'][:20]:20s} - {station['free_bikes']:2}/{station['boxes']}")

closest_citybikes("Arbeitergasse 5, Wien")

10 nearest Stations to Arbeitergasse 5, Wien:
_____________________
  1. [0.1 km] Siebenbrunnenplatz   - 18/26
  2. [0.4 km] Reinprechtsdorfer Br - 19/20
  3. [0.5 km] Arbeitergasse        - 24/26
  4. [0.7 km] Margaretengürtel U4  - 15/30
  5. [0.7 km] Matzleinsdorfer Plat -  8/24
  6. [0.8 km] Mittersteig          -  3/28
  7. [0.8 km] Pilgramgasse U4      - 18/21
  8. [0.9 km] Hartmanngasse        -  6/26
  9. [1.1 km] Gumpendorfer Gürtel  - 12/27
 10. [1.2 km] Falco Stiege         -  4/17


In [128]:
closest_citybikes(num_results=20)

20 nearest Stations to Stephansplatz, Wien, Österreich:
_______________________________
  1. [0.1 km] Singerstraße         - 21/22
  2. [0.3 km] Hoher Markt          - 25/26
  3. [0.4 km] Wallnerstraße        - 22/27
  4. [0.5 km] Schwedenplatz        -  9/30
  5. [0.7 km] Johannesgasse        -  9/20
  6. [0.7 km] Kärntner Ring        - 14/16
  7. [0.7 km] Oper                 - 18/23
  8. [0.7 km] Stadtpark Stubenring - 11/20
  9. [0.8 km] Julius Raab Platz    - 16/20
 10. [0.8 km] Schottenring U4      - 15/21
 11. [0.9 km] Rathausplatz         -  2/20
 12. [0.9 km] Universitätsring     -  8/38
 13. [0.9 km] Volksgarten          -  7/16
 14. [1.0 km] Karmeliterplatz      -  4/14
 15. [1.0 km] Obere Donaustraße    -  6/20
 16. [1.0 km] Wien Mitte           - 16/28
 17. [1.0 km] Treitlstraße         - 17/20
 18. [1.0 km] Sigmund Freud Park   - 17/36
 19. [1.1 km] Schwarzenbergplatz   -  6/20
 20. [1.1 km] Museumsplatz         - 28/36


In [127]:
closest_citybikes("Giefinggasse 6, Wien")

10 nearest Stations to Giefinggasse 6, Wien:
____________________
  1. [4.4 km] Millennium Tower     - 24/35
  2. [4.6 km] Friedrich Engels Pla - 22/27
  3. [5.0 km] Hellwagstraße        -  8/30
  4. [5.2 km] Traisengasse         - 17/24
  5. [5.6 km] Radingerstraße       - 28/33
  6. [5.7 km] Jägerstraße U6       -  8/26
  7. [6.0 km] Messeplatz           -  2/40
  8. [6.1 km] Wallensteinplatz     - 15/27
  9. [6.2 km] Krieau               - 10/24
 10. [6.3 km] Heinestraße          - 13/38



1. Get the water quality of Austrian lakes from here https://www.ages.at/typo3temp/badegewaesser_db.json and print all the latest available data for Viennese water bodies in a comprehensive form
1. Check out the Catalogue of Viennese Energy Lighthouse Projects: "https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:ENINNOVPRJV2OGD%20&srsName=EPSG:4326&outputFormat=json"
1. Browse https://www.data.gv.at/?s=json for more interesting datasets